<a href="https://colab.research.google.com/github/chihway/cosmology_on_beach_2022/blob/main/Tutorial2_measure_2pt_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Measure 2pt Functions

In this tutorial we will learn to measure 2pt functions from the samples we defined in the previous tutorial. 

We will learn to:
* Measure real-space two-point correlation functions  

In [1]:
import numpy as np

To perform a fast two-point measurement, we rely on a popular package called TreeCorr, from Mike Jarvis (scientist at UPenn). You can read about TreeCorr here. This package is used in papers like XXX. 

In [2]:
# you only need to install this once
!pip install treecorr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.8 MB 8.6 MB/s 


In [3]:
import treecorr